In [17]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from geoloc_functions import find_location,distance_to_centre, direction_from_centre
from openlocationcode import openlocationcode as olc
from pluscode_functions import convert_pluscode
from iris_functions import create_iris_polygon_dict, find_iris

In [18]:
# Read recipe inputs
new_inputs = dataiku.Dataset("new_inputs_editable")
new_inputs_df = new_inputs.get_dataframe()
iris_75 = dataiku.Dataset("iris_75")
iris_75_df = iris_75.get_dataframe()

/Users/nathanbry/Dataiku/dss-design/code-envs/python/real_estate/lib/python3.7/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


In [29]:
new_inputs_df['adresse_full'] = new_inputs_df['adresse']+', '+new_inputs_df['code_postal'].astype(str)

/Users/nathanbry/Dataiku/dss-design/code-envs/python/real_estate/lib/python3.7/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


In [31]:
new_inputs_df['latitude'] = find_location(new_inputs_df['adresse_full'])[0]
new_inputs_df['longitude'] = find_location(new_inputs_df['adresse_full'])[1]

/Users/nathanbry/Dataiku/dss-design/code-envs/python/real_estate/lib/python3.7/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


In [0]:
new_inputs_df

In [0]:
new_inputs_df['distance_to_centre']= distance_to_centre(new_inputs_df['latitude'],new_inputs_df['longitude'])

In [0]:
new_inputs_df['direction_from_centre'] = np.vectorize(direction_from_centre)(new_inputs_df['longitude'],new_inputs_df['latitude'])

In [0]:
new_inputs_df

In [0]:
new_inputs_df['pluscode_16_first'] = np.vectorize(olc.encode)(new_inputs_df['latitude'],new_inputs_df['longitude'],17)

In [0]:
new_inputs_df['pluscode_10'] = [code[:11] for code in new_inputs_df['pluscode_16_first'].to_list()]

In [0]:
new_inputs_df['v_coordinate'],new_inputs_df['h_coordinate'],new_inputs_df['reduced_v_coordinate'],new_inputs_df['reduced_h_coordinate'],new_inputs_df['reduced_coordinates_couple'] = convert_pluscode(new_inputs_df['pluscode_10'])

In [0]:
iris_coords_codes = create_iris_polygon_dict(iris_75_df['CODE_IRIS'],iris_75_df['Geo Shape'])

In [0]:
new_inputs_df['iris_code'] = find_iris(new_inputs_df['latitude'],new_inputs_df['longitude'],iris_coords_codes)

In [0]:
def correct_date(date_column):

    dates_list = date_column.to_list()
    new_dates_list = []

    for date in dates_list:
        if date > 2021:
            new_date = 2021
        elif date < 2014:
            new_date = 2014
        else:
            new_date = date

        new_dates_list.append(new_date)

    return new_dates_list

In [0]:
new_inputs_df['date_mutation_year'] = correct_date(new_inputs_df['date_mutation_year'])

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

inputs_located_df = new_inputs_df # For this sample code, simply copy input to output


# Write recipe outputs
inputs_located = dataiku.Dataset("inputs_located")
inputs_located.write_with_schema(inputs_located_df)